In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def fill_construction_year(construction_year):
    if construction_year == 0:
        return np.random.choice(a=list(year_prob_dict.keys()), p=list(year_prob_dict.values()), replace=True)
    return construction_year

In [3]:
df = pd.read_csv("../data/X_validate.csv")
display(df.head())
df.info()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
id                       14850 non-null int64
amount_tsh               14850 non-null float64
date_recorded            14850 non-null object
funder                   13981 non-null object
gps_height               14850 non-null int64
installer                13973 non-null object
longitude                14850 non-null float64
latitude                 14850 non-null float64
wpt_name                 14850 non-null object
num_private              14850 non-null int64
basin                    14850 non-null object
subvillage               14751 non-null object
region                   14850 non-null object
region_code              14850 non-null int64
district_code            14850 non-null int64
lga                      14850 non-null object
ward                     14850 non-null object
population               14850 non-null int64
public_meeting           14029 non-null object
r

In [4]:
f = open("../data/mean_dicts.pkl", 'rb')
coord_dict_region = pickle.load(f)
coord_dict_lga = pickle.load(f)
gps_dict_region = pickle.load(f)
gps_dict_district = pickle.load(f)
pop_dict_ward = pickle.load(f)
pop_dict_district = pickle.load(f)
year_prob_dict = pickle.load(f)
f.close()

In [5]:
# coordinates
for i, row in df.iterrows():
    if row['longitude'] < 25:
        region = row['region_code']
        row['longitude'] = coord_dict_region.get('longitude').get(region)
        row['latitude']= coord_dict_region.get('latitude').get(region)

df['longitude'].value_counts()

33.244879    269
33.091564    186
32.920579      2
39.080573      2
34.895027      2
            ... 
30.926134      1
37.227967      1
36.032819      1
35.628949      1
34.452772      1
Name: longitude, Length: 14392, dtype: int64

In [6]:
df['latitude'].value_counts() # check

-3.491473    269
-2.620502    186
-2.474560      2
-8.904674      2
-6.990042      2
            ... 
-9.320133      1
-9.114386      1
-3.134371      1
-3.885609      1
-8.477215      1
Name: latitude, Length: 14392, dtype: int64

In [7]:
# altitude
for i, row in df.iterrows():
    region = df['region_code'][i]
    district = df['district_code'][i]
    # lowest point is sea level so neg. numbers are suspect
    if df['gps_height'][i] <= 0:
        df['gps_height'][i] = gps_dict_region.get(region)
    # fill in remaining wiht distict (less complete)
    if df['gps_height'][i] == 0:
        df['gps_height'][i] = gps_dict_district.get(district)
df['gps_height'].value_counts() # check for 0s

56      1276
191      679
743      640
861      578
642      498
        ... 
195        1
179        1
2210       1
2202       1
2069       1
Name: gps_height, Length: 2105, dtype: int64

In [8]:
# time
df['date_recorded'] = pd.to_datetime(df['date_recorded'])
df['year_rec'] = pd.DatetimeIndex(df['date_recorded']).year
df['month_rec'] = pd.DatetimeIndex(df['date_recorded']).month
df['construction_year'] = df['construction_year'].apply(fill_construction_year)
df['age'] = df['year_rec'] - df['construction_year']
df['construction_year'].value_counts()[0:10]

2009    1019
2010     983
2008     967
2000     808
2006     606
2007     577
2011     499
2003     464
2004     453
2012     420
Name: construction_year, dtype: int64

In [9]:
# population
for i, row in df.iterrows():
    ward = df['ward'][i]
    district = df['district_code'][i]
    if df['population'][i] <= 0:
        df['population'][i] = pop_dict_ward.get(ward)
    if df['population'][i] <= 0:
        df['population'][i] = pop_dict_district.get(district)

df['population'].fillna(0, inplace=True)
        
display(df['population'].value_counts()) # check

1.0       1786
180.0     1052
182.0      821
229.0      818
132.0      710
          ... 
458.0        1
495.0        1
956.0        1
91.0         1
4735.0       1
Name: population, Length: 661, dtype: int64

In [10]:
# scheme
df['scheme_management'].fillna(value='unknown', inplace=True)
df['scheme_management'].isna().sum() # check

0

In [11]:
# permit/meeting
boolean = ['permit', 'public_meeting']
for col in boolean:
    df[col].fillna(False, inplace=True)
    df[col].astype(int)
    print(f"Value counts: {col} \n {df[col].value_counts()} \n -------")

Value counts: permit 
 True     9754
False    5096
Name: permit, dtype: int64 
 -------
Value counts: public_meeting 
 True     12738
False     2112
Name: public_meeting, dtype: int64 
 -------


In [12]:
# local government authority
for i, lga in enumerate(df['lga']):
    item = lga.lower()
    if 'rural' in item:
        df['lga'][i] = 'rural'
        continue
    if 'urban' in item:
        df['lga'][i] = 'urban'
    else:
        df['lga'][i] = 'other'
df['lga'].value_counts()

other    12153
rural     2262
urban      435
Name: lga, dtype: int64

In [13]:
# drop
to_drop_numeric = ['num_private', 'amount_tsh']
to_drop_cats = ['date_recorded', 'recorded_by', 'waterpoint_type_group', 'quantity_group', 
                   'quality_group', 'payment_type', 'management_group', 'scheme_name', 'wpt_name',
                   'funder', 'installer', 'extraction_type', 'extraction_type_class', 'subvillage',
                   'ward', 'source_class', 'source_type']
to_drop = to_drop_numeric + to_drop_cats
df.drop(columns=to_drop, inplace=True)

In [14]:
# check
display(df.info())
(df == 0).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 24 columns):
id                       14850 non-null int64
gps_height               14850 non-null int64
longitude                14850 non-null float64
latitude                 14850 non-null float64
basin                    14850 non-null object
region                   14850 non-null object
region_code              14850 non-null int64
district_code            14850 non-null int64
lga                      14850 non-null object
population               14850 non-null float64
public_meeting           14850 non-null bool
scheme_management        14850 non-null object
permit                   14850 non-null bool
construction_year        14850 non-null int64
extraction_type_group    14850 non-null object
management               14850 non-null object
payment                  14850 non-null object
water_quality            14850 non-null object
quantity                 14850 non-null object
sour

None

id                          0
gps_height                  4
longitude                   0
latitude                    0
basin                       0
region                      0
region_code                 0
district_code               4
lga                         0
population                 12
public_meeting           2112
scheme_management           0
permit                   5096
construction_year           0
extraction_type_group       0
management                  0
payment                     0
water_quality               0
quantity                    0
source                      0
waterpoint_type             0
year_rec                    0
month_rec                   0
age                       302
dtype: int64

In [17]:
# export for modelin
df.to_csv("../data/X_validate_transformed.csv", index=False)